<a href="https://colab.research.google.com/github/alaaguedda/python-Colab-Trainer/blob/main/oil_gas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import glob
import os
import re

In [ ]:
from google.colab import files
files.upload()  # Choose your kaggle.json

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d afrniomelo/3w-dataset
!unzip 3w-dataset.zip -d 3w_dataset


# 1) Get all parquet file paths in the target folder
parquet_files = glob.glob("3w_dataset/2.0.0/0/*.parquet")

dfs = []
row_limit = 150_000  # target number of rows
total_rows = 0

# 2) Loop through files until we hit row limit
for file in parquet_files:
    if total_rows >= row_limit:
        break

    # Extract well ID & timestamp from filename
    filename = os.path.basename(file)
    match = re.match(r"(.+?)_(\d+)\.parquet", filename)
    well_id = match.group(1) if match else None
    timestamp = match.group(2) if match else None

    # Read parquet file
    temp_df = pd.read_parquet(file)

    # Add extracted info
    temp_df["well_id"] = well_id
    temp_df["timestamp"] = timestamp

    dfs.append(temp_df)
    total_rows += len(temp_df)

# 3) Merge into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

# 4) Sort and reset index
df.sort_values(by=["well_id", "timestamp"], inplace=True)
df.reset_index(drop=True, inplace=True)

print(f"Loaded {len(df)} rows")
print(df.head())


#Here is the fetching area

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [ ]:

# Save the DataFrame as a parquet file in Google Drive
save_path = '/content/drive/MyDrive/3w_dataset_70k.parquet'
df.to_parquet(save_path, index=False)

print(f"Saved to {save_path}")


Saved to /content/drive/MyDrive/3w_dataset_70k.parquet


In [ ]:
  df = pd.read_parquet('/content/drive/MyDrive/3w_dataset_70k.parquet')


In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
df =df.dropna(axis=1 ,how='all')

In [ ]:
print(df.isna().mean().sort_values(ascending=False).head(30) *100)

In [ ]:
df.columns

Index(['ESTADO-DHSV', 'ESTADO-M1', 'ESTADO-M2', 'ESTADO-PXO', 'ESTADO-SDV-GL',
       'ESTADO-SDV-P', 'ESTADO-W1', 'ESTADO-W2', 'ESTADO-XO', 'P-ANULAR',
       'P-JUS-CKGL', 'P-MON-CKP', 'P-PDG', 'P-TPT', 'QGL', 'T-JUS-CKP',
       'T-PDG', 'T-TPT', 'class', 'state', 'well_id', 'timestamp'],
      dtype='object')

In [ ]:
rename_map = {
    'ESTADO-DHSV': 'Valve_Status',
    'ESTADO-M1': 'Module_1_Status',
    'ESTADO-M2': 'Module_2_Status',
    'ESTADO-PXO': 'Pump_XO_Status',
    'ESTADO-SDV-GL': 'Gas_Lift_Valve',
    'ESTADO-SDV-P': 'Production_Valve',
    'ESTADO-W1': 'Well_1_Status',
    'ESTADO-W2': 'Well_2_Status',
    'ESTADO-XO': 'XO_Status',
    'P-ANULAR': 'Annular_Pressure',
    'P-JUS-CKGL': 'Downstream_Gas_Pressure',
    'P-MON-CKP': 'Monitor_Pressure',
    'P-PDG': 'Downhole_Pressure',
    'P-TPT': 'Tubing_Pressure',
    'QGL': 'Gas_Lift_Rate',
    'T-JUS-CKP': 'Downstream_Temp',
    'T-PDG': 'Downhole_Temp',
    'T-TPT': 'Tubing_Temp',
    'class': 'Class',
    'state': 'State',
    'well_id': 'Well_ID',
    'timestamp': 'Timestamp'
}

df.rename(columns=rename_map, inplace=True)
df.columns

Index(['Valve_Status', 'Module_1_Status', 'Module_2_Status', 'Pump_XO_Status',
       'Gas_Lift_Valve', 'Production_Valve', 'Well_1_Status', 'Well_2_Status',
       'XO_Status', 'Annular_Pressure', 'Downstream_Gas_Pressure',
       'Monitor_Pressure', 'Downhole_Pressure', 'Tubing_Pressure',
       'Gas_Lift_Rate', 'Downstream_Temp', 'Downhole_Temp', 'Tubing_Temp',
       'Class', 'State', 'Well_ID', 'Timestamp'],
      dtype='object')